<a href="https://colab.research.google.com/github/YasserTuma/Redes_Neurais_Artificiais-A.A_2.2/blob/main/PP2_2_3_Validac%CC%A7a%CC%83o_Cruzada_e_Busca_em_Grade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Redes Neurais Artificiais 2021.1

- **Disciplina**: Redes Neurais Artificiais 2021.1  
- **Professora**: Elloá B. Guedes (ebgcosta@uea.edu.br)  
- **Github**: http://github.com/elloa
- **Integrantes**:

 *   Danilo Frazão (1715080036);
 *   Rafael Aragão Morais (1915080022);
 *   Yasser Tuma (1815310044) 
        

Levando em conta a base de dados **_Forest Cover Type_**, esta parte do Projeto Prático diz respeito à proposição e avaliação de múltiplas redes neurais artificiais do tipo feedforward multilayer perceptron para o problema da classificação multi-classe da cobertura florestal em uma área do Roosevelt National Forest.

## Busca em Grade

Uma maneira padrão de escolher os parâmetros de um modelo de Machine Learning é por meio de uma busca em grade via força bruta. O algoritmo da busca em grade é dado como segue:

1. Escolha a métrica de desempenho que você deseja maximizar  
2. Escolha o algoritmo de Machine Learning (exemplo: redes neurais artificiais). Em seguida, defina os parâmetros ou hiperparâmetros deste tipo de modelo sobre os quais você dseja otimizar (número de épocas, taxa de aprendizado, etc.) e construa um array de valores a serem testados para cada parâmetro ou hiperparâmetro.  
3. Defina a grade de busca, a qual é dada como o produto cartesiano de cada parâmetro a ser testado. Por exemplo, para os arrays [50, 100, 1000] e [10, 15], tem-se que a grade é [(50,10), (50,15), (100,10), (100,15), (1000,10), (1000,15)].
4. Para cada combinação de parâmetros a serem otimizados, utilize o conjunto de treinamento para realizar uma validação cruzada (holdout ou k-fold) e calcule a métrica de avaliação no conjunto de teste (ou conjuntos de teste)
5. Escolha a combinação de parâmetros que maximizam a métrica de avaliação. Este é o modelo otimizado.

Por que esta abordagem funciona? Porque a busca em grade efetua uma pesquisa extensiva sobre as possíveis combinações de valores para cada um dos parâmetros a serem ajustados. Para cada combinação, ela estima a performance do modelo em dados novos. Por fim, o modelo com melhor métrica de desempenho é escolhido. Tem-se então que este modelo é o que melhor pode vir a generalizar mediante dados nunca antes vistos.

## Efetuando a Busca em Grade sobre Hiperparâmetros das Top-6 RNAs

Considerando a etapa anterior do projeto prático, foram identificadas pelo menos 6 melhores Redes Neurais para o problema da classificação multi-classe da cobertura florestal no conjunto de dados selecionado. Algumas destas redes possuem atributos categóricos como variáveis preditoras, enquanto outras possuem apenas os atributos numéricos como preditores.

A primeira etapa desta segunda parte do projeto consiste em trazer para este notebook estas seis arquiteturas, ressaltando:

1. Número de neurônios ocultos por camada  
2. Função de Ativação  
3. Utilização ou não de atributos categóricos   
4. Desempenho médio +- desvio padrão nos testes anteriores  
5. Número de repetições que a equipe conseguiu realizar para verificar os resultados  

Elabore uma busca em grade sobre estas arquiteturas que contemple variações nos hiperparâmetros a seguir, conforme documentação de [MLPClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)

A. Solver  (Não usar o LBFGS, pois é mais adequado para datasets pequenos)  
B. Batch Size  
C. Learning Rate Init  
D. Paciência (n_iter_no_change)  
E. Épocas  

Nesta busca em grande, contemple a utilização do objeto [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)

### Importando as bibliotecas e o dataset

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from joblib import dump, load
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data= pd.read_csv("/content/drive/My Drive/Colab Notebooks/RNA/covtype.csv")
data

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Soil_Type1,Soil_Type2,Soil_Type3,Soil_Type4,Soil_Type5,Soil_Type6,Soil_Type7,Soil_Type8,Soil_Type9,Soil_Type10,Soil_Type11,Soil_Type12,Soil_Type13,Soil_Type14,Soil_Type15,Soil_Type16,Soil_Type17,Soil_Type18,Soil_Type19,Soil_Type20,Soil_Type21,Soil_Type22,Soil_Type23,Soil_Type24,Soil_Type25,Soil_Type26,Soil_Type27,Soil_Type28,Soil_Type29,Soil_Type30,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2396,153,20,85,17,108,240,237,118,837,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
581008,2391,152,19,67,12,95,240,237,119,845,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
581009,2386,159,17,60,7,90,236,241,130,854,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
581010,2384,170,15,60,5,90,230,245,143,864,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3


### Obtendo o atributo alvo e os atributos preditores

In [ ]:
# Obtendo o atributo-alvo
def getTarget(data, column):
  return data[column]

data_y = getTarget(data, "Cover_Type")


In [ ]:
def getPredictors(data, columns):
  return data.drop(columns, axis=1)


data_x = getPredictors(data, "Cover_Type")

In [ ]:
def standardization(value, mean, std):
  if(std != 0):
    return (value - mean) / std
  else:
    return value

def applyStandardization(set):
  for column in set.head(0).columns:
    mean = set[column].mean()
    std = set[column].std()
    set[column] = set[column].apply(standardization, args=(mean, std))
  return set

data_x = applyStandardization(data_x)

### Busca em grade

In [ ]:
parameter_space = {
    'solver': ['sgd', 'adam'],
    'batch_size': [100, 200],
    'learning_rate_init': [0.1, 0.4],
    'n_iter_no_change': [5, 10],
    'max_iter': [100, 300],  
}

In [ ]:
def testing(MLP, test_x, test_y):
  predictions = MLP.predict(test_x.values)

  print("\nMatriz de confusao:\n")
  print(confusion_matrix(test_y, predictions))

  print("\nAcuracia:\n")
  print(MLP.score(test_x, test_y))

  print("\nPrecisao:\n")
  print(precision_score(test_y, predictions, average='micro'))

  print("\nRevocacao:\n")
  print(recall_score(test_y, predictions, average='micro'))

  print("\nF-Score:\n")
  print(f1_score(test_y, predictions, average='micro'))

## Validação Cruzada k-fold

Na elaboração da busca em grid, vamos avaliar os modelos propostos segundo uma estratégia de validação cruzada ainda não explorada até o momento: a validação cruzada k-fold. Segundo a mesma, o conjunto de dados é particionado em k partes: a cada iteração, separa-se uma das partes para teste e o modelo é treinado com as k-1 partes remanescentes. Valores sugestivos de k na literatura são k = 3, 5 ou 10, pois o custo computacional desta validação dos modelos é alto. A métrica de desempenho é resultante da média dos desempenhos nas k iterações. A figura a seguir ilustra a ideia desta avaliação

<img src = "https://ethen8181.github.io/machine-learning/model_selection/img/kfolds.png" width=600></img>

Considerando a métrica de desempenho F1-Score, considere a validação cruzada 5-fold para aferir os resultados da busca em grande anterior.

### Primeira rede neural

* Arquitetura propia;   
* Função de ativação: tangente hiperbolica;
* Duas camadas ocultas, cada uma com 20 neuronios.

---

* Melhor Classificador:

 *   bath_size = 100
 *   learning_rate_init = 0.1
 *   max_iter = 300
 *   n_iter_no_change = 5

*   Pontuação do melhor classificador: 0.5908482398016645

In [ ]:
MLP_1 = MLPClassifier(hidden_layer_sizes=(20, 20,), activation='tanh')

MLP_1 = GridSearchCV(MLP_1, parameter_space, cv=5, verbose=3, scoring='f1_micro')

In [ ]:
MLP_1.fit(data_x, data_y)

Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV 1/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=sgd;, score=0.582 total time= 2.0min
[CV 2/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=sgd;, score=0.404 total time= 1.4min
[CV 3/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=sgd;, score=0.392 total time= 1.6min
[CV 4/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=sgd;, score=0.501 total time= 2.1min
[CV 5/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=sgd;, score=0.536 total time= 1.1min
[CV 1/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=adam;, score=0.552 total time=  23.3s
[CV 2/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=adam;, score=0.548 total time=  23.6s
[CV 3/5] END batch_

GridSearchCV(cv=5,
             estimator=MLPClassifier(activation='tanh',
                                     hidden_layer_sizes=(20, 20)),
             param_grid={'batch_size': [100, 200],
                         'learning_rate_init': [0.1, 0.4],
                         'max_iter': [100, 300], 'n_iter_no_change': [5, 10],
                         'solver': ['sgd', 'adam']},
             scoring='f1_micro', verbose=3)

In [ ]:
MLP_1.best_estimator_

MLPClassifier(activation='tanh', batch_size=100, hidden_layer_sizes=(20, 20),
              learning_rate_init=0.1, max_iter=300, n_iter_no_change=5)

In [ ]:
MLP_1.best_score_

0.5908482398016645

### Segunda rede neural

* Arquitetura propia;   
* Função de ativação: tangente hiperbolica;
* Duas camadas ocultas, cada uma com 15 neuronios.

---

* Melhor Classificador:

 *   bath_size = 100
 *   learning_rate_init = 0.1
 *   max_iter = 300
 *   n_iter_no_change = 5

*   Pontuação do melhor classificador: 0.5888567512904601

In [ ]:
MLP_2 = MLPClassifier(hidden_layer_sizes=(15,15), activation='tanh')

MLP_2 = GridSearchCV(MLP_2, parameter_space, cv=5, verbose=3, scoring='f1_micro')

In [ ]:
MLP_2.fit(data_x, data_y)

Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV 1/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=sgd;, score=0.593 total time= 1.6min
[CV 2/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=sgd;, score=0.448 total time=  55.4s
[CV 3/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=sgd;, score=0.478 total time= 1.1min
[CV 4/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=sgd;, score=0.543 total time= 2.4min
[CV 5/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=sgd;, score=0.527 total time= 1.6min
[CV 1/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=adam;, score=0.597 total time=  25.4s
[CV 2/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=adam;, score=0.588 total time=  25.6s
[CV 3/5] END batch_

GridSearchCV(cv=5,
             estimator=MLPClassifier(activation='tanh',
                                     hidden_layer_sizes=(15, 15)),
             param_grid={'batch_size': [100, 200],
                         'learning_rate_init': [0.1, 0.4],
                         'max_iter': [100, 300], 'n_iter_no_change': [5, 10],
                         'solver': ['sgd', 'adam']},
             scoring='f1_micro', verbose=3)

In [ ]:
MLP_2.best_estimator_

MLPClassifier(activation='tanh', batch_size=100, hidden_layer_sizes=(15, 15),
              learning_rate_init=0.1, max_iter=300, n_iter_no_change=5)

In [ ]:
MLP_2.best_score_

0.5888567512904601

### Terceira rede neural

* Arquitetura propia;   
* Função de ativação: função sigmoide;
* Uma camada oculta com 30 neuronios.

---

* Melhor Classificador:

 *   bath_size = 100
 *   learning_rate_init = 0.4
 *   max_iter = 300
 *   n_iter_no_change = 5

*   Pontuação do melhor classificador: 0.6091334340178542


In [ ]:
MLP_3 = MLPClassifier(hidden_layer_sizes=(30,), activation='logistic')

MLP_3 = GridSearchCV(MLP_3, parameter_space, cv=5, verbose=3, scoring='f1_micro')

In [ ]:
MLP_3.fit(data_x, data_y)

Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV 1/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=sgd;, score=0.595 total time= 3.1min
[CV 2/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=sgd;, score=0.436 total time= 3.1min
[CV 3/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=sgd;, score=0.414 total time= 2.2min
[CV 4/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=sgd;, score=0.478 total time= 2.2min
[CV 5/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=sgd;, score=0.558 total time= 2.8min
[CV 1/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=adam;, score=0.528 total time= 2.1min
[CV 2/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=adam;, score=0.441 total time=  57.9s
[CV 3/5] END batch_

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


[CV 1/5] END batch_size=200, learning_rate_init=0.1, max_iter=100, n_iter_no_change=10, solver=sgd;, score=0.587 total time= 4.9min
[CV 2/5] END batch_size=200, learning_rate_init=0.1, max_iter=100, n_iter_no_change=10, solver=sgd;, score=0.380 total time= 4.7min
[CV 3/5] END batch_size=200, learning_rate_init=0.1, max_iter=100, n_iter_no_change=10, solver=sgd;, score=0.411 total time= 4.1min


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


[CV 4/5] END batch_size=200, learning_rate_init=0.1, max_iter=100, n_iter_no_change=10, solver=sgd;, score=0.483 total time= 4.9min
[CV 5/5] END batch_size=200, learning_rate_init=0.1, max_iter=100, n_iter_no_change=10, solver=sgd;, score=0.553 total time= 3.8min
[CV 1/5] END batch_size=200, learning_rate_init=0.1, max_iter=100, n_iter_no_change=10, solver=adam;, score=0.558 total time= 1.7min
[CV 2/5] END batch_size=200, learning_rate_init=0.1, max_iter=100, n_iter_no_change=10, solver=adam;, score=0.445 total time= 1.5min
[CV 3/5] END batch_size=200, learning_rate_init=0.1, max_iter=100, n_iter_no_change=10, solver=adam;, score=0.475 total time= 2.0min
[CV 4/5] END batch_size=200, learning_rate_init=0.1, max_iter=100, n_iter_no_change=10, solver=adam;, score=0.546 total time= 1.6min
[CV 5/5] END batch_size=200, learning_rate_init=0.1, max_iter=100, n_iter_no_change=10, solver=adam;, score=0.531 total time= 1.9min
[CV 1/5] END batch_size=200, learning_rate_init=0.1, max_iter=300, n_it

0.6091334340178542

In [ ]:
MLP_3.best_estimator_

MLPClassifier(activation='logistic', batch_size=100, hidden_layer_sizes=(30,),
              learning_rate_init=0.4, max_iter=300, n_iter_no_change=5)

In [ ]:
MLP_3.best_score_

0.6091334340178542

### Quarta rede neural

* Arquitetura da regra da piramide geometrica;   
* Função de ativação: tangente hiperbolica;
* Duas camadas ocultas, a primeira com 10 neuronios e a segunda com 15 neuronios.

---

* Melhor Classificador:

 *   bath_size = 100
 *   learning_rate_init = 0.1
 *   max_iter = 300
 *   n_iter_no_change = 5

*   Pontuação do melhor classificador: 0.5884782513090315


In [ ]:
MLP_4 = MLPClassifier(hidden_layer_sizes=(10, 15,), activation='tanh')

MLP_4 = GridSearchCV(MLP_4, parameter_space, cv=5, verbose=3, scoring='f1_micro')

In [ ]:
MLP_4.fit(data_x, data_y)

Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV 1/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=sgd;, score=0.591 total time= 1.6min
[CV 2/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=sgd;, score=0.446 total time= 1.3min
[CV 3/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=sgd;, score=0.461 total time=  50.6s
[CV 4/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=sgd;, score=0.570 total time=  43.6s
[CV 5/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=sgd;, score=0.524 total time=  57.5s
[CV 1/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=adam;, score=0.596 total time=  24.5s
[CV 2/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=adam;, score=0.671 total time=  25.2s
[CV 3/5] END batch_

GridSearchCV(cv=5,
             estimator=MLPClassifier(activation='tanh',
                                     hidden_layer_sizes=(10, 15)),
             param_grid={'batch_size': [100, 200],
                         'learning_rate_init': [0.1, 0.4],
                         'max_iter': [100, 300], 'n_iter_no_change': [5, 10],
                         'solver': ['sgd', 'adam']},
             scoring='f1_micro', verbose=3)

In [ ]:
MLP_4.best_estimator_

MLPClassifier(activation='tanh', batch_size=100, hidden_layer_sizes=(10, 15),
              learning_rate_init=0.1, max_iter=300)

In [ ]:
MLP_4.best_score_

0.5884782513090315

### Quinta rede neural

* Arquitetura da regra da piramide geometrica;   
* Função de ativação: tangente hiperbolica;
* Uma camada oculta com 25 neuronios.

---

* Melhor Classificador:

 *   bath_size = 200
 *   learning_rate_init = 0.4
 *   max_iter = 100
 *   n_iter_no_change = 5

*   Pontuação do melhor classificador: 0.5848174323724369


In [ ]:
MLP_5 = MLPClassifier(hidden_layer_sizes=(25,), activation='tanh')

MLP_5 = GridSearchCV(MLP_5, parameter_space, cv=5, verbose=3, scoring='f1_micro')

In [ ]:
MLP_5.fit(data_x, data_y)

Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV 1/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=sgd;, score=0.540 total time= 2.0min
[CV 2/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=sgd;, score=0.418 total time= 3.1min
[CV 3/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=sgd;, score=0.421 total time= 2.4min
[CV 4/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=sgd;, score=0.526 total time= 2.3min
[CV 5/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=sgd;, score=0.557 total time= 1.4min
[CV 1/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=adam;, score=0.574 total time= 1.1min
[CV 2/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=adam;, score=0.602 total time= 1.8min
[CV 3/5] END batch_

0.5848174323724369

In [ ]:
MLP_5.best_estimator_

MLPClassifier(activation='tanh', batch_size=200, hidden_layer_sizes=(25,),
              learning_rate_init=0.4, max_iter=100)

In [ ]:
MLP_5.best_score_

0.5848174323724369

### Sexta rede neural

* Arquitetura da regra da piramide geometrica;   
* Função de ativação: sigmoide;
* Uma camada oculta com 25 neuronios.

---

* Melhor Classificador:

 *   bath_size = 200
 *   learning_rate_init = 0.4
 *   max_iter = 100
 *   n_iter_no_change = 5

*   Pontuação do melhor classificador: 0.6212658337769308

In [ ]:
MLP_6 = MLPClassifier(hidden_layer_sizes=(25,), activation='logistic')

MLP_6 = GridSearchCV(MLP_6, parameter_space, cv=5, verbose=3, scoring='f1_micro')

In [ ]:
MLP_6.fit(data_x, data_y)

Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV 1/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=sgd;, score=0.576 total time= 1.5min
[CV 2/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=sgd;, score=0.394 total time= 2.9min
[CV 3/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=sgd;, score=0.436 total time= 2.0min
[CV 4/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=sgd;, score=0.481 total time= 1.7min
[CV 5/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=sgd;, score=0.556 total time= 3.5min
[CV 1/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=adam;, score=0.590 total time=  58.1s
[CV 2/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=adam;, score=0.469 total time= 1.0min
[CV 3/5] END batch_

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


[CV 4/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=10, solver=sgd;, score=0.514 total time= 4.0min
[CV 5/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=10, solver=sgd;, score=0.563 total time= 3.0min
[CV 1/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=10, solver=adam;, score=0.593 total time= 1.5min
[CV 2/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=10, solver=adam;, score=0.455 total time= 2.3min
[CV 3/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=10, solver=adam;, score=0.486 total time= 1.5min
[CV 4/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=10, solver=adam;, score=0.561 total time= 1.2min
[CV 5/5] END batch_size=100, learning_rate_init=0.1, max_iter=100, n_iter_no_change=10, solver=adam;, score=0.557 total time= 1.7min
[CV 1/5] END batch_size=100, learning_rate_init=0.1, max_iter=300, n_it

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


[CV 3/5] END batch_size=200, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=sgd;, score=0.448 total time= 4.5min
[CV 4/5] END batch_size=200, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=sgd;, score=0.511 total time= 3.7min
[CV 5/5] END batch_size=200, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=sgd;, score=0.561 total time= 3.3min
[CV 1/5] END batch_size=200, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=adam;, score=0.577 total time= 1.0min
[CV 2/5] END batch_size=200, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=adam;, score=0.524 total time=  43.7s
[CV 3/5] END batch_size=200, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=adam;, score=0.471 total time=  58.1s
[CV 4/5] END batch_size=200, learning_rate_init=0.1, max_iter=100, n_iter_no_change=5, solver=adam;, score=0.519 total time= 1.2min
[CV 5/5] END batch_size=200, learning_rate_init=0.1, max_iter=100, n_iter_no_ch

...

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


[CV 4/5] END batch_size=200, learning_rate_init=0.1, max_iter=100, n_iter_no_change=10, solver=sgd;, score=0.500 total time= 4.5min


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


[CV 5/5] END batch_size=200, learning_rate_init=0.1, max_iter=100, n_iter_no_change=10, solver=sgd;, score=0.542 total time= 4.5min
[CV 1/5] END batch_size=200, learning_rate_init=0.1, max_iter=100, n_iter_no_change=10, solver=adam;, score=0.592 total time= 1.9min
[CV 2/5] END batch_size=200, learning_rate_init=0.1, max_iter=100, n_iter_no_change=10, solver=adam;, score=0.414 total time= 2.5min
[CV 3/5] END batch_size=200, learning_rate_init=0.1, max_iter=100, n_iter_no_change=10, solver=adam;, score=0.478 total time= 2.6min
[CV 4/5] END batch_size=200, learning_rate_init=0.1, max_iter=100, n_iter_no_change=10, solver=adam;, score=0.537 total time= 1.8min
[CV 5/5] END batch_size=200, learning_rate_init=0.1, max_iter=100, n_iter_no_change=10, solver=adam;, score=0.542 total time= 2.1min
[CV 1/5] END batch_size=200, learning_rate_init=0.1, max_iter=300, n_iter_no_change=5, solver=sgd;, score=0.604 total time= 2.3min
[CV 2/5] END batch_size=200, learning_rate_init=0.1, max_iter=300, n_ite

0.6212658337769308

In [ ]:
MLP_6.best_estimator_

MLPClassifier(activation='logistic', batch_size=100, hidden_layer_sizes=(25,),
              learning_rate_init=0.4, max_iter=300)

In [ ]:
MLP_6.best_score_

0.6212658337769308

In [ ]:
dump(MLP_6, 'MLP_6.joblib')

['MLP_6.joblib']

## Identificando a mellhor solução

Como resultado da busca em grande com validação cruzada 5-fold, identifique o modelo otimizado com melhor desempenho para o problema. Apresente claramente este modelo, seus parâmetros, hiperparâmetros otimizados e resultados para cada um dos folds avaliados. Esta é a melhor solução identificada em decorrência deste projeto



> O modelo com o melhor desempenho foi o sexto modelo, ele obteve a pontuação de 0.6212658337769308, foi a maior entre os 6 modelos que foram treinados.
Seus parâmetros são:

* a função de ativação tangente hiperbolica; 
* uma camada oculta com 25 neuronios. 

> Os melhores hiperparâmetros otimizados deste modelo são:

*   bath_size = 200;
*   learning_rate_init = 0.4;
*   max_iter = 100;
*   n_iter_no_change = 5.



## Empacotando a solução

Suponha que você deve entregar este classificador ao órgão responsável por administrar o Roosevelt National Park. Para tanto, você deve fazer uma preparação do mesmo para utilização neste cenário. Uma vez que já identificou os melhores parâmetros e hiperparâmetros, o passo remanescente consiste em treinar o modelo com estes valores e todos os dados disponíveis, salvando o conjunto de pesos do modelo ao final para entrega ao cliente. Assim, finalize o projeto prático realizando tais passos.

1. Consulte a documentação a seguir:
https://scikit-learn.org/stable/modules/model_persistence.html  
2. Treine o modelo com todos os dados  
3. Salve o modelo em disco  
4. Construa uma rotina que recupere o modelo em disco  
5. Mostre que a rotina é funcional, fazendo previsões com todos os elementos do dataset e exibindo uma matriz de confusão das mesmas

In [ ]:
MLP = load('MLP_6.joblib')

In [ ]:
testing(MLP, data_x, data_y)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  "X does not have valid feature names, but"



Matriz de confusao:

[[143784  60717      8      0      0      0   7331]
 [ 52003 225263   5263      0      0      7    765]
 [     0   6796  28958      0      0      0      0]
 [     0      0   2747      0      0      0      0]
 [     4   9225    264      0      0      0      0]
 [    14   6207  11146      0      0      0      0]
 [  6368    138      0      0      0      0  14004]]

Acuracia:

0.7091230473725155

Precisao:

0.7091230473725155

Revocacao:

0.7091230473725155

F-Score:

0.7091230473725155
